In [6]:
from elasticsearch import Elasticsearch, helpers
import pandas as pd
from tqdm import tqdm
import time
import json
import re

def get_doc_id(context):
    for k,v in wiki.items():
        if v['text'] == context:
            return k


class elastic:
    def __init__(self, INDEX_NAME, context_path="../data/wikipedia_documents.json"):
        self.index_name = INDEX_NAME
        try:
            self.es.transport.close()
        except:
            pass
        self.context_path = context_path
        config = {
            "host": "localhost", 
            "port": 9200,
            "timeout": 100,
            "max_retries": 10,
            "retry_on_timeout": True,
            }
        self.es = Elasticsearch([config])
        
        self.index_setting = {
            "settings": {
                "index": {
                    "analysis": {
                        "analyzer": {
                            "korean": {
                                "type": "custom",
                                "tokenizer": "nori_tokenizer",
                                "filter": ["shingle"],
                            }
                        }
                    }
                }
            },
            "mappings": {
                "properties": {
                    "text": {
                        "type": "text",
                        "analyzer": "korean",
                        "search_analyzer": "korean",
                    },
                    "title": {
                        "type": "text",
                        "analyzer": "korean",
                        "search_analyzer": "korean",
                    },
                    # "document_id": {
                    #     "type": "integer",
                    # }
                }
            },
        }

    def build_elatic(self):
        with open(self.context_path) as file:
            json_data = json.load(file)
        docs = []
        for i, j in json_data.items():
            docs.append(
                {
                    "_index": "wikipedia",
                    "_source": {
                        "text": j["text"], 
                        "title": j["title"],
                        # "document_id": i
                        },
                }
            )

        if self.es.indices.exists(self.index_name):
            print(self.es.indices)
            print(self.index_name)
            pass
        else:
            self.es.indices.create(index=self.index_name, body=self.index_setting)
            helpers.bulk(self.es, docs)

    def retrieve(self,query_or_dataset,topk):
        datas = []
        for i in tqdm(range(len(query_or_dataset))):
            cp = {i: v for i, v in query_or_dataset[i].items()}
            if "context" in query_or_dataset[i].keys() and "answers" in query_or_dataset[i].keys():
                cp['original_context'] = query_or_dataset[i]['context']

            query = query_or_dataset[i]['question']
            query = query.replace('/', '')
            query = query.replace('~', ' ')
            res = self.es.search(index=self.index_name, q=query, size=topk)
            hits = res['hits']['hits']
            context = []
            score = []
            document_id = []
            for docu in hits:
                # print(docu)
                context.append(docu['_source']['text'])
                score.append(docu['_score'])
                document_id.append(get_doc_id(docu['_source']['text']))
            # score = list(map(lambda x: str(x/sum(score)),score))
            cp['context'] = '///'.join(context)#리스트를 사용하려면 join없이 그냥 context를 쓰면 됩니다.
            cp['score'] = score
            cp['document_id'] = document_id
            datas.append(cp)

        return pd.DataFrame(datas)

    def retrieve_false(self, query_or_dataset, topk):
        datas = []
        for i in tqdm(range(len(query_or_dataset))):
            cp = {i: v for i, v in query_or_dataset[i].items()}
            if (
                "context" in query_or_dataset[i].keys()
                and "answers" in query_or_dataset[i].keys()
            ):
                cp["original_context"] = query_or_dataset[i]["context"]

            query = query_or_dataset[i]["question"]
            query = query.replace("/", "")
            query = query.replace("~", " ")
            res = self.es.search(index=self.index_name, q=query, size=topk+1)
            x = res["hits"]["hits"]
            context = []
            for docu in x:
                cont = docu["_source"]["text"]
                # groud truth 제거
                if cont == cp["original_context"]: continue
                context.append(docu["_source"]["text"])
            context = context[:topk]
            cp["context"] = "///".join(context)
            datas.append(cp)

        return pd.DataFrame(datas)

    def get_relevant_doc(self, query: str, topk:int):
        query = query.replace('/', '')
        query = query.replace('~', ' ')
        res = self.es.search(index=self.index_name, q=query, size=topk)
        hits = res['hits']['hits']
        context = []
        for docu in hits:
            context.append(docu['_source']['text'])
        join_context = '<다음 문맥>'.join(context)
        return join_context

In [7]:
from datasets import load_from_disk

dataset = load_from_disk("../data/train_dataset")
train_datasets = dataset["train"]
valid_datasets = dataset["validation"]

In [8]:
x = elastic("wikipedia")
x.build_elatic()


wikipedia


<ipython-input-6-2d713e7bacb4>:80: DeprecationWarning: Using positional arguments for APIs is deprecated and will be disabled in 8.0.0. Instead use only keyword arguments for all APIs. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  if self.es.indices.exists(self.index_name):
/opt/conda/lib/python3.8/site-packages/elasticsearch/connection/base.py:209: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.15/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


## Train dataset retrieving for hard negatives

In [9]:
# p = x.retrieve_false(train_datasets, 100)
# p.to_csv("/opt/ml/data/train_elastic_top100_noanswer.csv")

## Validation dataset retrieving for hard negatives

In [10]:
# val = x.retrieve_false(valid_datasets, 100)
# val.to_csv("/opt/ml/data/valid_elastic_top100_noanswer.csv")

In [11]:
# p.context[0][:1000]

## Test dataset retrieval for elastic search

In [12]:
""" remove same items """
import json
context_path = '../data/wikipedia_documents.json'
with open(context_path, 'r', encoding= "utf-8") as f:
    wiki = json.load(f)
search_corpus = list(dict.fromkeys([v['text'] for v in wiki.values()]))
search_corpus[0]

'이 문서는 나라 목록이며, 전 세계 206개 나라의 각 현황과 주권 승인 정보를 개요 형태로 나열하고 있다.\n\n이 목록은 명료화를 위해 두 부분으로 나뉘어 있다.\n\n# 첫 번째 부분은 바티칸 시국과 팔레스타인을 포함하여 유엔 등 국제 기구에 가입되어 국제적인 승인을 널리 받았다고 여기는 195개 나라를 나열하고 있다.\n# 두 번째 부분은 일부 지역의 주권을 사실상 (데 팍토) 행사하고 있지만, 아직 국제적인 승인을 널리 받지 않았다고 여기는 11개 나라를 나열하고 있다.\n\n두 목록은 모두 가나다 순이다.\n\n일부 국가의 경우 국가로서의 자격에 논쟁의 여부가 있으며, 이 때문에 이러한 목록을 엮는 것은 매우 어렵고 논란이 생길 수 있는 과정이다. 이 목록을 구성하고 있는 국가를 선정하는 기준에 대한 정보는 "포함 기준" 단락을 통해 설명하였다. 나라에 대한 일반적인 정보는 "국가" 문서에서 설명하고 있다.'

In [13]:
query_or_dataset = {int(k):v for k,v in wiki.items()}
print(query_or_dataset[0].keys())
query_or_dataset[0]

dict_keys(['text', 'corpus_source', 'url', 'domain', 'title', 'author', 'html', 'document_id'])


{'text': '이 문서는 나라 목록이며, 전 세계 206개 나라의 각 현황과 주권 승인 정보를 개요 형태로 나열하고 있다.\n\n이 목록은 명료화를 위해 두 부분으로 나뉘어 있다.\n\n# 첫 번째 부분은 바티칸 시국과 팔레스타인을 포함하여 유엔 등 국제 기구에 가입되어 국제적인 승인을 널리 받았다고 여기는 195개 나라를 나열하고 있다.\n# 두 번째 부분은 일부 지역의 주권을 사실상 (데 팍토) 행사하고 있지만, 아직 국제적인 승인을 널리 받지 않았다고 여기는 11개 나라를 나열하고 있다.\n\n두 목록은 모두 가나다 순이다.\n\n일부 국가의 경우 국가로서의 자격에 논쟁의 여부가 있으며, 이 때문에 이러한 목록을 엮는 것은 매우 어렵고 논란이 생길 수 있는 과정이다. 이 목록을 구성하고 있는 국가를 선정하는 기준에 대한 정보는 "포함 기준" 단락을 통해 설명하였다. 나라에 대한 일반적인 정보는 "국가" 문서에서 설명하고 있다.',
 'corpus_source': '위키피디아',
 'url': 'TODO',
 'domain': None,
 'title': '나라 목록',
 'author': None,
 'html': None,
 'document_id': 0}

In [14]:
import pandas as pd
import numpy as np
import json

# load test dataset as dataframe
test = load_from_disk('../data/test_dataset')
test_datasets = test['validation']
test_datasets

Dataset({
    features: ['id', 'question'],
    num_rows: 600
})

In [15]:
test_datasets[0]

{'question': "유령'은 어느 행성에서 지구로 왔는가?", 'id': 'mrc-1-000653'}

In [16]:
test = x.retrieve(test_datasets, 100)

100%|██████████| 600/600 [07:09<00:00,  1.40it/s]


In [20]:
test.head(2)

,question,id,context,score,document_id
0,유령'은 어느 행성에서 지구로 왔는가?,mrc-1-000653,목성의 대기에서 보이는 줄무늬는 적도와 평행하면서 행성을 둘러싸는 대(zone)와 ...,"[20.489973, 20.10779, 19.855621, 19.532476, 18...","[43280, 47081, 35064, 24024, 42242, 10509, 978..."
1,용병회사의 경기가 좋아진 것은 무엇이 끝난 이후부터인가?,mrc-1-001113,SK에서 방출된 이후 그는 일본을 여행하며 잠시 신변을 정리하고 있었다. 그 때 좌...,"[23.782045, 21.42108, 21.085941, 20.75889, 19....","[20081, 23179, 15556, 13590, 18470, 50337, 291..."


In [21]:
# split context column values with ///
df_test = test.copy()
df_test['context'] = df_test['context'].str.split('///')
df_test.head(2)

,question,id,context,score,document_id
0,유령'은 어느 행성에서 지구로 왔는가?,mrc-1-000653,[목성의 대기에서 보이는 줄무늬는 적도와 평행하면서 행성을 둘러싸는 대(zone)와...,"[20.489973, 20.10779, 19.855621, 19.532476, 18...","[43280, 47081, 35064, 24024, 42242, 10509, 978..."
1,용병회사의 경기가 좋아진 것은 무엇이 끝난 이후부터인가?,mrc-1-001113,[SK에서 방출된 이후 그는 일본을 여행하며 잠시 신변을 정리하고 있었다. 그 때 ...,"[23.782045, 21.42108, 21.085941, 20.75889, 19....","[20081, 23179, 15556, 13590, 18470, 50337, 291..."


In [22]:
df_test.to_csv("/opt/ml/data/test_elastic_top100.csv")

In [19]:
# check whether document id is same as original document id
for num, (i, j) in enumerate(wiki.items()) :
    if int(i) != j['document_id'] :
        print('안돼!!')
        